## 使用 Faiss 和 DeepSeek 构建 RAG
DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Faiss 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

In [1]:
!pip install sentence-transformers faiss-cpu python-dotenv deepseek-sdk tiktoken

In [2]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = "sk-aad29a173cf541a79e4a54f8ce153c8c"
print(api_key)

sk-aad29a173cf541a79e4a54f8ce153c8c


### 加载并分割《民法典》文档

In [3]:
from glob import glob
import re

def split_civil_code(file_path="mfd.md"):
    """
    按《民法典》的“**第xxx条**”格式分割文档，返回条款列表
    
    参数:
        file_path: 民法典markdown文件路径，默认"mfd.md"
    返回:
        list: 每个元素为完整条款（含标题和内容）
    """
    try:
        # 读取文件内容
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
        
        # 优化正则：
        # 1. 匹配“**第X条**”（支持中文数字和阿拉伯数字，如“第一条”“第100条”）
        # 2. 保留条款标题作为分割标识
        pattern = r'(\*\*第[零一二三四五六七八九十百千\d]+条\*\*)'
        sections = re.split(pattern, content)
        
        # 组合条款（标题+内容），过滤空字符串
        result = []
        # 从索引1开始，每2个元素一组（标题+内容）
        for i in range(1, len(sections), 2):
            title = sections[i].strip()  # 条款标题（如“**第一条**”）
            content = sections[i+1].strip() if (i+1 < len(sections)) else ""  # 条款内容
            # 跳过空内容的条款
            if title:
                result.append(f"{title}\n{content}")
        
        print(f"✅ 成功分割《民法典》，共提取 {len(result)} 条条款")
        return result
    
    except FileNotFoundError:
        print(f"❌ 未找到文件：{file_path}")
        return []
    except Exception as e:
        print(f"❌ 分割失败：{str(e)}")
        return []

# 执行分割
text_lines = split_civil_code()

# 示例：打印前3条验证结果
if text_lines:
    print("\n示例条款：")
    for i in range(min(3, len(text_lines))):
        print(f"\n条款{i+1}：\n{text_lines[i][:200]}...")  # 只显示前200字符

✅ 成功分割《民法典》，共提取 387 条条款

示例条款：

条款1：
**第二百零四条**
为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。...

条款2：
**第二百零五条**
本编调整因物的归属和利用产生的民事关系。...

条款3：
**第二百零六条**
国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。
国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。
国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。...


### 初始化 faiss 向量存储并插入数据

In [4]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [5]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()


In [6]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [7]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.02752976  0.0608853   0.00388525 -0.00215193 -0.02774976 -0.0118618
 -0.04020916 -0.06023417 -0.03813156  0.0100272 ]


In [8]:
from sentence_transformers import SentenceTransformer

# 加载中文嵌入模型
embedding_model = SentenceTransformer("shibing624/text2vec-base-chinese")

# 生成嵌入向量（这一步会定义 doc_embeddings）
doc_embeddings = embedding_model.encode(text_lines)  # text_lines 是分割后的条款列表

In [9]:
import faiss
import numpy as np

class FaissVectorStore:
    def __init__(self):
        self.index = None
        self.texts = []
        self.collection_exists = False

    def create_collection(self, dimension):
        self.index = faiss.IndexFlatIP(dimension)  # 内积距离
        self.collection_exists = True

    def insert(self, vectors, texts):
        if not self.collection_exists:
            raise ValueError("请先创建集合！")
        vectors = np.array(vectors, dtype=np.float32)
        self.index.add(vectors)
        self.texts.extend(texts)

    def search(self, query_vector, top_k=3):
        if not self.collection_exists:
            raise ValueError("请先创建集合并插入数据！")
        query_vector = np.array([query_vector], dtype=np.float32)
        distances, indices = self.index.search(query_vector, top_k)
        return [(self.texts[i], distances[0][idx]) for idx, i in enumerate(indices[0]) if i < len(self.texts)]

# 初始化并插入数据（此时 doc_embeddings 已存在）
vector_store = FaissVectorStore()
# 使用 doc_embeddings 的维度创建集合（shape[1] 表示向量的维度）
vector_store.create_collection(dimension=doc_embeddings.shape[1])
# 插入向量和对应的文本（text_lines 是分割后的条款）
vector_store.insert(doc_embeddings, text_lines)

### 构建提示词并调用 DeepSeek 大模型

In [13]:
import os
import requests

# 1. 配置 API Key 和请求参数
API_KEY = "sk-aad29a173cf541a79e4a54f8ce153c8c"  # 你的 API Key
API_URL = "https://api.deepseek.com/v1/chat/completions"  # DeepSeek API 地址

# 2. 定义系统提示和用户提示（和你原逻辑一致）
SYSTEM_PROMPT = """你是一名专业的法律助手，需严格依据提供的《中华人民共和国民法典》相关内容回答问题，禁止编造信息。"""

def build_user_prompt(question, context):
    return f"""
    <context>
    {context}
    </context>

    <question>
    {question}
    </question>

    请根据上述<context>中的内容，用中文回答<question>中的问题，要求回答准确、简洁，引用具体的民法典条款（若有）。
    """

# 3. 示例问题 + 检索上下文（和你原逻辑一致）
question = "民法典中关于自然人隐私权的保护是如何规定的？"
query_embedding = embedding_model.encode([question])[0]
similar_docs = vector_store.search(query_embedding, top_k=3)
context = "\n\n".join([doc for doc, _ in similar_docs])
USER_PROMPT = build_user_prompt(question, context)

# 4. 手动发送 HTTP 请求调用 DeepSeek API
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}
payload = {
    "model": "deepseek-chat",
    "messages": [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ],
    "temperature": 0.2  # 控制回答稳定性，法律问题建议设低（0.1-0.3）
}

# 发送请求并获取结果
try:
    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status()  # 若请求失败（如 API Key 无效），抛出异常
    result = response.json()
    ai_answer = result["choices"][0]["message"]["content"]
except Exception as e:
    ai_answer = f"调用 API 失败：{str(e)}"

# 5. 打印结果（和你原逻辑一致）
print("问题：", question)
print("检索到的相关内容：")
for doc, score in similar_docs:
    print(f"相似度：{score:.4f}\n{doc}\n")
print("AI 回答：", ai_answer)

问题： 民法典中关于自然人隐私权的保护是如何规定的？
检索到的相关内容：
相似度：180.8743
**第二百六十三条**
私人对其合法所有的财产，依法享有占有、使用、收益和处分的权利。

相似度：176.0773
**第二百六十八条**
法律对集体所有的财产有特别保护的，依照其规定。

相似度：175.3439
**第二百六十四条**
国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。

AI 回答： 根据您提供的上下文内容，未包含与自然人隐私权保护相关的法律条款。

您引用的《中华人民共和国民法典》第二百六十三条、第二百六十四条和第二百六十八条仅涉及财产权保护的相关规定，未涉及隐私权保护内容。建议查阅民法典人格权编中关于隐私权保护的专门条款获取准确信息。


### 测试一下

In [14]:
# 1. 定义测试问题列表
test_questions = [
    "民法典中关于夫妻共同财产的规定在第几条？",
    "自然人下落不明满几年可以申请宣告失踪？",
    "民法典规定多少岁以上为成年人？"
]

# 2. 批量执行测试
for idx, question in enumerate(test_questions, 1):
    print(f"=== 测试用例 {idx}：{question} ===")
    # 检索相关条款
    query_embedding = embedding_model.encode([question])[0]
    similar_docs = vector_store.search(query_embedding, top_k=3)
    # 拼接上下文
    context = "\n\n".join([doc for doc, _ in similar_docs])
    # 生成回答
    user_prompt = build_user_prompt(question, context)
    response = deepseek_client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]
    )
    # 打印结果
    print("检索到的相关条款：")
    for doc, score in similar_docs:
        print(f"- 相似度：{score:.4f}\n{doc[:150]}...")  # 显示前150字符
    print("AI 回答：", response.choices[0].message.content, "\n")
    

=== 测试用例 1：民法典中关于夫妻共同财产的规定在第几条？ ===
检索到的相关条款：
- 相似度：177.0313
**第三百二十一条**
共有物分割后，各共有人对其分得的财产享有所有权。
共有物分割后，各共有人对分割前的共有债务，承担连带责任。...
- 相似度：176.4286
**第三百一十二条**
共同共有人对共有的不动产或者动产共同享有所有权。
共同共有人对共有财产享有平等的占有、使用、收益和处分的权利。...
- 相似度：174.9416
**第三百二十二条**
共有物分割后，各共有人对分割前的共有债务，承担连带责任。
但是，共有物分割后，各共有人对其分得的财产，仅以其分得的财产为限承担责任。...
AI 回答： 根据您提供的《中华人民共和国民法典》上下文内容，未包含关于夫妻共同财产的具体条款。建议查阅民法典婚姻家庭编相关条文获取准确信息。 

=== 测试用例 2：自然人下落不明满几年可以申请宣告失踪？ ===
检索到的相关条款：
- 相似度：171.7724
**第三百四十九条**
遗失物自发布招领公告之日起一年内无人认领的，归国家所有。...
- 相似度：166.9332
**第二百三十五条**
遗失物自发布招领公告之日起一年内无人认领的，归国家所有。...
- 相似度：156.7214
**第五百四十一条**
撤销权自债权人知道或者应当知道撤销事由之日起一年内行使。自债务人的行为发生之日起五年内没有行使撤销权的，该撤销权消灭。...
AI 回答： 根据您提供的《中华人民共和国民法典》条款内容，未包含关于宣告失踪期限的具体规定。建议您查阅《中华人民共和国民法典》第四十条等相关条款获取准确信息。 

=== 测试用例 3：民法典规定多少岁以上为成年人？ ===
检索到的相关条款：
- 相似度：151.1135
**第二百五十五条**
国家所有财产的范围和管理方式，由法律、行政法规规定。...
- 相似度：149.3454
**第五百六十三条**
当事人可以约定一方解除合同的事由。解除合同的事由发生时，解除权人可以解除合同。...
- 相似度：148.7723
**第五百七十六条**
损害赔偿的范围，包括因违约造成的损失，以及合同履行后可以获得的利益；但是，不得超过违约方订立合同时预见到或者应当预见到的因违约可能造成的损失。...
AI 回答： 

In [15]:
def civil_code_qa(question, top_k=3):
    """
    民法典 RAG 问答函数
    参数：question（用户问题）、top_k（检索条款数量）
    返回：dict（包含问题、检索结果、AI回答）
    """
    # 检索
    query_embedding = embedding_model.encode([question])[0]
    similar_docs = vector_store.search(query_embedding, top_k=top_k)
    # 拼接上下文
    context = "\n\n".join([doc for doc, _ in similar_docs])
    # 生成回答
    user_prompt = build_user_prompt(question, context)
    response = deepseek_client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]
    )
    # 整理结果
    return {
        "question": question,
        "retrieved_docs": [{"content": doc, "similarity": round(score, 4)} for doc, score in similar_docs],
        "ai_answer": response.choices[0].message.content
    }

# 使用示例
result = civil_code_qa("合同到期后未续签，但继续履行原合同内容，该合同是否有效？")
print("问题：", result["question"])
print("检索结果：")
for doc in result["retrieved_docs"]:
    print(f"- 相似度：{doc['similarity']}\n{doc['content'][:200]}...")
print("AI 回答：", result["ai_answer"])

问题： 合同到期后未续签，但继续履行原合同内容，该合同是否有效？
检索结果：
- 相似度：207.7136993408203
**第五百零四条**
当事人对合同的效力可以约定附期限。
附生效期限的合同，自期限届至时生效。
附终止期限的合同，自期限届满时终止。...
- 相似度：207.0670928955078
**第五百六十六条**
合同解除后，尚未履行的，终止履行；已经履行的，根据履行情况和合同性质，当事人可以请求恢复原状或者采取其他补救措施，并有权请求赔偿损失。
合同因违约解除的，解除权人可以请求违约方承担违约责任。...
- 相似度：205.13070678710938
**第四百九十六条**
承诺生效时合同成立。
当事人采用合同书形式订立合同的，自双方当事人签字或者盖章时合同成立。...
AI 回答： 根据《中华人民共和国民法典》第五百零四条第二款规定，附终止期限的合同自期限届满时终止。合同到期后未续签，原合同因期限届满而终止。

但若双方继续履行原合同内容，可能构成事实合同关系，需根据实际履行情况认定新的合同关系成立。具体效力需结合双方意思表示及实际履行行为综合判断，但原合同因期限届满已终止。


### 搭建简单交互界面### 创建 civil_code_rag.py 文件

In [16]:
!pip install streamlit

In [24]:
# 页面配置
st.title("《民法典》RAG 问答助手")
question = st.text_input("请输入你的问题（如：民法典中关于隐私权的规定？）")

if st.button("获取答案"):
    if question:
        with st.spinner("正在检索并生成回答..."):
            result = civil_code_qa(question)
            st.subheader("你的问题")
            st.write(result["question"])
            st.subheader("检索到的相关条款")
            for i, doc in enumerate(result["retrieved_docs"], 1):
                st.write(f"**{i}. 相似度：{doc['similarity']}**")
                st.write(doc["content"])
            st.subheader("AI 回答")
            st.write(result["ai_answer"])
    else:
        st.warning("请输入问题后再点击获取答案！")

2025-09-13 20:10:25.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 20:10:25.588 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 20:10:25.589 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 20:10:25.589 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 20:10:25.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 20:10:25.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 20:10:25.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-13 20:10:25.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar